# Práctica #6: Bayes Ingenuo

Profesor: Luis Norberto Zúñiga Morales, Universidad Iberoamericana Ciudad de México

Instrucciones:

- Realizar cada una de las actividades propuestas en este libro de Google Colab.
- Se puede realizar en equipos de 2 personas.
- Para la entrega de sus respuestas, es importante que la celda en cuestión se encuentre ejecutada para mostrar su resultados.
- Descargar su libro con terminación IPYNB y subirlo en la actividad correspondiente de Brigthspace.


Importamos las librerías que usaremos para nuestro programa.

Nombres:

In [ ]:
from math import sqrt
from math import pi
from math import exp

## Primer paso: preparar los datos para el algoritmo

Lo primero que hacemos es una función que nos separa los datos en clases. Recurden que esto es importante ya que NB funciona alrededor de los ejemplos observados en cada clase de nuestro conjunto de datos. En esta práctica, cada dato tiene la forma:

[$x_1$, $x_2$, clase]

Además, vamos a guardar cada punto del conjunto de datos en la variable `dataset`, que es una lista de listas.

In [ ]:
dataset = [[3.393533211,2.331273381,0],
	[3.110073483,1.781539638,0],
	[1.343808831,3.368360954,0],
	[3.582294042,4.67917911,0],
	[2.280362439,2.866990263,0],
	[7.423436942,4.696522875,1],
	[5.745051997,3.533989803,1],
	[9.172168622,2.511101045,1],
	[7.792783481,3.424088941,1],
	[7.939820817,0.791637231,1]]

*(3 puntos)* Vamos a crear una función especial cuyo objetivo es separar los datos en dos clases: los de la clase `0` y los de la clase `1`. En el celda de código de abajo, crear una función que tome como argumentos el dataset (una lista de listas) y regrese un diccionario con la siguiente estructura:



```
{ 0: [[x1, x2, clase], [x1, x2, clase],...]
1: [[x1, x2, clase], [x1, x2, clase],...]
}
```


In [ ]:
def separar_clases(dataset):
	separated = dict()
	# to-do: terminar el código
	return separated

Como guía, revisen el diccionario de abajo que resulta al mandar llamar la función `separar_clases`.

In [ ]:
print(separar_clases(dataset))

{0: [[3.393533211, 2.331273381, 0], [3.110073483, 1.781539638, 0], [1.343808831, 3.368360954, 0], [3.582294042, 4.67917911, 0], [2.280362439, 2.866990263, 0]], 1: [[7.423436942, 4.696522875, 1], [5.745051997, 3.533989803, 1], [9.172168622, 2.511101045, 1], [7.792783481, 3.424088941, 1], [7.939820817, 0.791637231, 1]]}


## Segundo paso: estadísticas del modelo

El segundo paso consiste en determinar unas estadísticas para nuestro modelo. En particular, vamos a suponer que los datos siguien una distribución $N∼(\mu,\sigma^2)$, por lo que necesitamos estimar los parámetros correspondiente usando nuestro conjunto de datos con las siguientes fórmulas:

* $μ=\frac{1}{N}\sum_{i=1}^N x_i$

* $\sigma = \sqrt{\frac{\sum_{i=1}^N x_i – \mu(x)^2}{N-1}}$

donde $N$ indica el número de elementos del conjunto de datos.

*(4 puntos)* Crear dos funciones que calculen la media y la desviación estándar de un conjunto de puntos. Cada función debe regresar un escalar. Más adelante las vamos a usar.

In [ ]:
# Calcula la media de una lista de numeros
def media(numeros):
	return # to-do: terminar el código

In [ ]:
# Calcula la desviacion estandar de una lista de numeros
def stdev(numeros):
	return # to-do: terminar el código

Necesitamos determinar la media y desviación estándar de cada característica que modela nuestros datos, es decir, si la primera características es representada por $x_1$ y tenemos 5 datos, necesitamos determinar la media y desviación estándar de los puntos
$$\{x_1^{(1)}, x_1^{(2)}, x_1^{(3)}, x_1^{(4)}, x_1^{(5)}\}$$

lo cual se puede interpretar como calcular las estadísticas a nivel columna en nuestro conjunto de datos.

*(2 puntos)* Crear una función llamada `resumen_dataset` que pida como argumento el `dataset` y calcule la media, desviación estándar y conteo de elementos en cada columna. Como resultado, debe regresar un lista de tuplas, donde cada tupla corresponde a la media, desviación estándar y conteo de elementos de cada columna:



```
[(5.178333386499999, 2.7665845055177263, 10), (2.9984683241, 1.218556343617447, 10)]
```



In [ ]:
# calcula la media, desviacion estandar y el conteo de cada columna del dataset
def resumen_dataset(dataset):
	# to-do: terminar el código
	return resultados

In [ ]:
print(resumen_dataset(dataset))

[(5.178333386499999, 2.7665845055177263, 10), (2.9984683241, 1.218556343617447, 10)]


Vamos a organizar el código un poco más usando todo lo anterior al crear una función que solicite la lista con las estadísticas anteriores y la vamos a meter en un diccionario para su uso posterior, con la particularidad de tener toda la información necesaria en un formato adecuado. Noten que los estadísticas son por clase (0 y 1) y para cada columna de características.

In [ ]:
# Separar el dataset por clase y calcular las estadisticas para cada fila
def resumen_por_clase(dataset):
  # separamos el dataset por clases, 0 y 1
	separated = separar_clases(dataset)
	summaries = dict()
  # se calculan las estadisticas para cada subconjunto
	for class_value, rows in separated.items(): # itera sobre la llave (clase) y valor (vector de datos)
		summaries[class_value] = resumen_dataset(rows)
	return summaries

In [ ]:
print(resumen_por_clase(dataset))

{0: [(2.7420144012, 0.9265683289298018, 5), (3.0054686692, 1.1073295894898725, 5)], 1: [(7.6146523718, 1.2344321550313704, 5), (2.9914679790000003, 1.4541931384601618, 5)]}


## Tercer paso: construcción de la función de probabilidad del modelo

La siguiente fase consiste en construir nuestra función de probabilidad para cade clase con la media y varianza que determinamos en los puntos anteriores. Vamos a programar la función normal:

$$f(x) = \frac{1}{\sqrt{2\pi}\sigma} \exp({-\frac{(x-\mu)^2}{(2 * sigma^2)}})$$

In [ ]:
# Calcular la funcion de distribucion gaussiana
def calcular_probabilidad(x, media, stdev):
	exponente = exp(-((x-media)**2 / (2 * stdev**2 )))
	return (1 / (sqrt(2 * pi) * stdev)) * exponente

Para calcular las probabilidades de cada clase, vamos a utilizar todo lo que hemos hecho hasta el momento. La probabilidad de que un dato pertenezca a una clase se determina por:

$$P(clase|x) = P(x|clase) * P(clase)$$

Utilizando el supuesto ingenuo (independencia entre las características que modelan los datos), tenemos que:

$$P(clase=0|x_1,x_2) = P(x_1|clase=0)  P(x_2|clase=0)  P(clase=0)$$

Es decir, necesitamos trabajar cada columna por separado ($x_1$ y $x_2$) y, además, para cada clase (0 y 1).

1. Primero, el número total de datos de entrenamiento se calcula a partir del conteo almacenado en las estadísticas de resumen para cada clase. Esto se usa en el cálculo de la probabilidad de una clase dada o $P(clase)$ como la proporción de filas con una clase dada de todas las filas en los datos de entrenamiento.

2. A continuación, se calculan las probabilidades para cada valor de entrada en la fila utilizando la función de densidad de probabilidad gaussiana y las estadísticas para esa columna y de esa clase. Las probabilidades se multiplican juntas a medida que se acumulan.

3. Este proceso se repite para cada clase en el conjunto de datos.

4. Finalmente, se devuelve un diccionario de probabilidades con una entrada para cada clase.

In [ ]:
# Calcular las probabilidad de cada clase para cada fila del dataset
def calcular_probabilidades_clase(resumen, fila):
	filas_totales = sum([resumen[label][0][2] for label in resumen])
	# print([resumen[label][0][2] for label in resumen])
	# print('filas totales', filas_totales)
	probs = dict() #diccionario que guarda las probabilidades
	for class_value, class_summaries in resumen.items(): #llave: valor de la clase, valor: estadisticos de cada columna
		probs[class_value] = resumen[class_value][0][2]/float(filas_totales) #5/10, P(CLASE=0)
		for i in range(len(class_summaries)): # itera sobre x_1 y x_2
			media, stdev, _ = class_summaries[i] # extrae los estadisticos
			probs[class_value] *= calcular_probabilidad(fila[i], media, stdev) # i=0 , calcula la prob. gaussiana P(x_1|clase=0)
	return probs

In [ ]:
# mostramos los resultados
resumen = resumen_por_clase(dataset)
# print('resumen:',resumen)
probs = calcular_probabilidades_clase(resumen, dataset[1])
print(probs)

{0: 0.03891229878967956, 1: 4.025712313269427e-05}


## Cuarto paso: realizar las predicciones con el modelo entrenado

Vamos a crear una función para predecir la clase de un nuevo dato:

In [ ]:
# Make prediction - highest probability is the prediction
def predict(resumen, nuevo_dato):
    probs = calcular_probabilidades_clase(resumen, nuevo_dato)
    best_label, best_prob = None, -1
    for class_val, prob in probs.items():
        if best_label is None or prob > best_prob:
            best_prob = prob
            best_label = class_val
    return best_label

In [ ]:
predict(resumen_por_clase(dataset), dataset[1])

0

*(2 puntos)*: Crear una función que, dado un conjunto de puntos, determine su clase usando la función `predict()` anterior.

In [ ]:
def mass_predict(resumen, datos):
  # to-do: terminar código

## Actividad

*(8 puntos)*: Usar el código anterior para entrenar un clasificador Bayesiano Ingenuo con el famoso dataset de Iris. Verificar que se haga lo siguiente:

- Descargar o importar el dataset (como les sea más conveniente). Explicar en una celda de código sobre qué trata el conjunto de datos.
- Ajustar los datos al formato que maneja el código en esta práctica.
- Separar los datos en conjuntos de prueba y entrenamiento. Entrenar el modelo con la partición de entrenamiento, y realizar las predicciones con la partición de prueba.
- Evaluar su modelo usando la medida F1, ya sea que ustedes la definan o puede utilizar cualquier otra definida en una librería.
- Reportar y explicar sus hallazgos.